In [ ]:
from imgaug import augmenters as iaa
import numpy as np
from imantics import Polygons, Mask
from skimage import *
from skimage import io
import skimage
import json
import os
import copy
from PIL import Image

In [ ]:
annotation_path = "C:/Users/David.So/Downloads/labels_train.json"
annotation_dump_path "C:/Users/David.So/Downloads/labels_train_aug.json"
file_path = "C:/Users/David.So/Downloads/train/train/"
disease = 'hr'

annotations = json.load(open(annotation_path))
annotations_upt = json.load(open(annotation_path))


for k,v in annotations.items():
    image = io.imread(file_path+v['filename'])
    height, width = image.shape[:2]
    
    if image.ndim != 3:
        image = skimage.color.gray2rgb(image)
        # If has an alpha channel, remove it for consistency
    if image.shape[-1] == 4:
        image = image[..., :3]
    if v['regions']:
        #initialisiere regions_vek, der später in neue json geladen wird
        regions = []
        # lade alle shapes von interesse
        shapes = [r['shape_attributes'] for r in v['regions']
                 if r["region_attributes"]["shape"] == disease]
        
        # augmentation -- 1-4 der aufgeführten Augmentations wird durchgeführt
        augmentation = iaa.SomeOf((1, 4), [
            iaa.Fliplr(0.5),
            iaa.Crop(percent=(0, 0.1)), 
            iaa.Multiply((0.8, 1.2), per_channel=0.2),
            iaa.Affine(
                scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                rotate=(-25, 25),
                shear=(-8, 8)
            )
            ], random_order=True)
        det = augmentation.to_deterministic()
        image_shape = image.shape
        image_aug = det.augment_image(image)
        assert image_aug.shape == image_shape, "Augmentation shouldn't change image size"
        
        for p in shapes:
            #erzeuge leere binary mask
            mask = np.zeros([height, width],
                dtype=np.uint8)
            
            #nur zur Nachvollziehbarkeit
            print(v['filename'])
            print(p['all_points_x'])
            print(p['all_points_y'])
            
            #fülle binary mask an entsprechenden stellen
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc] = 1

            mask_shape = mask.shape
            
            #augmentiere mask
            mask_aug = det.augment_image(mask)

            #Image.fromarray(mask*255).show()
            #Image.fromarray(mask_aug*255).show()
            #io.imshow(image)
            #io.imshow(image_aug)

            assert mask_aug.shape == mask_shape, "Augmentation shouldn't change mask size"
            
            
            #konvertiere binary mask zurück zu polygon
            polygons = Mask(mask_aug).polygons()
            print(polygons.points)   
            print("................................")

            #bringe polygon von [[x1,y1],[x2,y2],...] Struktur zu all_points_x, all_points_y Struktur
            x = []
            y = []


            if len(polygons.points) == 1:
                for i in polygons.points[0].tolist():
                    x.append(i[0])
                    y.append(i[1])

                region = {
                    "region_attributes": {"shape": disease
                                        },
                    "shape_attributes": {"all_points_x": x,
                                        "all_points_y": y,
                                        "name": "polygon"
                                        }
                        }
        
                regions.append(region)
        
        if len(regions) > 0:  
            #alles abspeichern
            annotations_upt[v['filename'].split(".")[0]+"_1."+v['filename'].split(".")[-1]] = {"file_attributes": {},
                                   "filename": v['filename'].split(".")[0]+"_1."+v['filename'].split(".")[-1],
                                   "regions": regions,
                                   "size": v['size']}
            with open(annotation_dump_path, 'w') as f:
                json.dump(annotations_upt, f)
                
            #io.imsave(file_path+v['filename']+"_1", image_aug)
            #io.imshow(image_aug)
            Image.fromarray(image_aug).save(file_path+v['filename'].split(".")[0]+"_1."+v['filename'].split(".")[-1])
                
                
                    
                

# Achtung

Damit Via das ganze annimmt muss das Feld size der augmentierten Bilder in der json stimmen, leider weiß ich nicht, wie VIA die Size berechnet...
Workaround: augmentierte Bilder in Via laden, annotations direkt als json exportieren und dann nächsten Chunk ausführen.

In [ ]:
annotations = json.load(open("C:/Users/David.So/Downloads/via_export_json.json"))

for k, v in annotations.items():
    annotations_upt[k] = annotations_upt[v['filename']]
    annotations_upt[k]['size'] = v['size']
    del annotations_upt[v['filename']]
    
with open("C:/Users/David.So/Downloads/labels_train_aug.json", 'w') as f:
                json.dump(annotations_upt, f)